nlp

## Librerías necesarias

In [1]:
# que no se impriman info y warnings
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' 

In [13]:
import tensorflow as tf
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
from tensorflow.keras import layers,callbacks,models,Sequential,losses
import seaborn as sns
from sklearn.metrics import confusion_matrix
#import keras_tuner
from tensorflow import keras
from keras import backend as K
import os,random
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator
#import tensorflow_hub as hub
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.efficientnet import preprocess_input
from tensorflow.keras.layers.experimental import preprocessing
import seaborn as sns
from sklearn.metrics import classification_report, confusion_matrix
import csv
from datetime import datetime
from sklearn.model_selection import train_test_split 

In [3]:
# Download helper functions script
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

--2023-12-15 14:20:49--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolviendo raw.githubusercontent.com (raw.githubusercontent.com)... 2606:50c0:8003::154, 2606:50c0:8001::154, 2606:50c0:8002::154, ...
Conectando con raw.githubusercontent.com (raw.githubusercontent.com)[2606:50c0:8003::154]:443... conectado.
Petición HTTP enviada, esperando respuesta... 200 OK
Longitud: 10246 (10K) [text/plain]
Guardando como: ‘helper_functions.py.1’

helper_functions.py 100%[===================>]  10,01K  --.-KB/s    en 0s      

2023-12-15 14:20:51 (28,5 MB/s) - ‘helper_functions.py.1’ guardado [10246/10246]



In [4]:
# Import series of helper functions for the notebook
from helper_functions import unzip_data, create_tensorboard_callback, plot_loss_curves, compare_historys

## Carga de datos

In [5]:
# Download data (same as from Kaggle)
!wget "https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip"

# Unzip data
unzip_data("nlp_getting_started.zip")

--2023-12-15 14:20:51--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolviendo storage.googleapis.com (storage.googleapis.com)... 2800:3f0:4002:80b::201b, 2800:3f0:4002:80d::201b, 2800:3f0:4002:801::201b, ...
Conectando con storage.googleapis.com (storage.googleapis.com)[2800:3f0:4002:80b::201b]:443... conectado.
Petición HTTP enviada, esperando respuesta... 200 OK
Longitud: 607343 (593K) [application/zip]
Guardando como: ‘nlp_getting_started.zip.4’

nlp_getting_started 100%[===================>] 593,11K  --.-KB/s    en 0,1s    

2023-12-15 14:20:52 (5,36 MB/s) - ‘nlp_getting_started.zip.4’ guardado [607343/607343]



In [6]:
!ls

08_introduction_to_nlp_in_tensorflow_video.ipynb  nlp_getting_started.zip.4
helper_functions.py				  __pycache__
helper_functions.py.1				  sample_submission.csv
nlp_getting_started.zip				  test.csv
nlp_getting_started.zip.1			  train.csv
nlp_getting_started.zip.2			  Untitled.ipynb
nlp_getting_started.zip.3


Unzipping nlp_getting_started.zip gives the following 3 .csv files:

    sample_submission.csv - an example of the file you'd submit to the Kaggle competition of your model's predictions.
    train.csv - training samples of real and not real diaster Tweets.
    test.csv - testing samples of real and not real diaster Tweets.



## Visualizando un dataset de texto

In [7]:
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [8]:

# viendo los datos anteriores puede deducirse que la clase positiva
# es un desastre es la clase "1"


# Shuffle training dataframe
train_df_shuffled = train_df.sample(frac=1, random_state=42) 
train_df_shuffled.head()

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [9]:
# How many examples of each class?

# sirve para ver si está balanceado

train_df.target.value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [10]:
# How many total samples?
len(train_df), len(test_df)

(7613, 3263)

In [11]:
random_index = random.randint(0, len(train_df)-5) # create random indexes not higher than the total number of samples
for row in train_df_shuffled[["text", "target"]][random_index:random_index+5].itertuples():
  _, text, target = row
  print(f"Target: {target}", "(real diaster)" if target > 0 else "(not real diaster)")
  print(f"Text:\n{text}\n")
  print("---\n")

Target: 1 (real diaster)
Text:
President Barack Obama has on air meltdown over opposition to Iran nuclear deal http://t.co/c0t7RvoTKj via @examinercom

---

Target: 0 (not real diaster)
Text:
My dad is panicking as my weight loss means he needs to hurry up with my new clothes fundwhen I reach my goal.  ??

---

Target: 1 (real diaster)
Text:
Anyone missing their license plate? Two stolen ones found on terrorist's car... http://t.co/CWGCciw3V6

---

Target: 0 (not real diaster)
Text:
@D33munni @JeanNamibian noooooooo ... *proceeds to fall off a cliff*

---

Target: 1 (real diaster)
Text:
Haunting memories drawn by survivors http://t.co/pRAro2OWia

---



### Split data into training and validation sets

We want to be able to see how our model is performing on unseen data whilst it trains.

And because the testing dataset doesn't have labels, we'll have to create a validation dataset to evaluate on (the model won't see the validation dataset during training so we can use its samples and labels to evaluate our model's performance).

In [14]:
# Use train_test_split to split training data into training and validation sets
train_sentences, val_sentences, train_labels, val_labels = train_test_split(train_df_shuffled["text"].to_numpy(),
                                                                            train_df_shuffled["target"].to_numpy(),
                                                                            test_size=0.1, # use 10% of training data for validation split
                                                                            random_state=42)

In [15]:
# Check the lengths
len(train_sentences), len(train_labels), len(val_sentences), len(val_labels)

(6851, 6851, 762, 762)

## Converting text into numbers

When dealing with a text problem, one of the first things you'll have to do before you can build a model is to convert your text to numbers.

There are a few ways to do this, namely:
* Tokenziation - direct mapping of token (a token could be a word or a character) to number
* Embedding - create a matrix of feature vector for each token (the size of the feature vector can be defined and this embedding can be learned)

### Text vectorization (tokenization)

https://www.tensorflow.org/api_docs/python/tf/keras/layers/TextVectorization

In [16]:
train_sentences[:5]

array(['@mogacola @zamtriossu i screamed after hitting tweet',
       'Imagine getting flattened by Kurt Zouma',
       '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
       "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
       'Somehow find you and I collide http://t.co/Ee8RpOahPk'],
      dtype=object)

In [20]:
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

# Use the default TextVectorization parameters
text_vectorizer = TextVectorization(max_tokens=100, # how many words in the vocabulary (automatically add <OOV>)
                                    standardize="lower_and_strip_punctuation",
                                    split="whitespace",
                                    ngrams=None, # create groups of n-words?
                                    output_mode="int", # how to map tokens to numbers
                                    output_sequence_length=None, # how long do you want your sequences to be?
                                    pad_to_max_tokens=True)

In [21]:
len(train_sentences[0].split())

7

In [22]:
# Find the average number of tokens (words) in the training tweets
round(sum([len(i.split()) for i in train_sentences])/len(train_sentences))

15

In [23]:
# Setup text vectorization variables
max_vocab_length = 10000 # max number of words to have in our vocabulary
max_length = 15 # max length our sequences will be (e.g. how many words from a Tweet does a model see?)

text_vectorizer = TextVectorization(max_tokens=max_vocab_length,
                                    output_mode="int",
                                    output_sequence_length=max_length)

In [24]:
# Fit the text vectorizer instance to the training data using the adapt() method
text_vectorizer.adapt(train_sentences)

In [25]:
# Create a sample sentence and tokenize it
sample_sentence = "There's a flood in my street!" # 6 palabras
text_vectorizer([sample_sentence]) # veo abajo que hace un pad con ceros para completar el largo de 15

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[264,   3, 232,   4,  13, 698,   0,   0,   0,   0,   0,   0,   0,
          0,   0]])>

In [28]:
# Create a sample sentence and tokenize it
sample_sentence = "another sentence"
text_vectorizer([sample_sentence])

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[165,   1,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0]])>

In [26]:
# Get the unique words in the vocabulary
words_in_vocab = text_vectorizer.get_vocabulary()
top_5_words = words_in_vocab[:5] # the most common words in the vocab
bottom_5_words = words_in_vocab[-5:] # the least common words in the vocab
print(f"Most common words in vocab: {top_5_words}")
print(f"Least common words in vocab: {bottom_5_words}")

Most common words in vocab: ['', '[UNK]', 'the', 'a', 'in']
Least common words in vocab: ['pages', 'paeds', 'pads', 'padres', 'paddytomlinson1']


In [27]:
# how long is our vocab?
len(words_in_vocab)

10000

## Creating an Embedding using an Embedding Layer

In [30]:
max_length,

(15,)

In [31]:
from tensorflow.keras import layers 

embedding = layers.Embedding(input_dim=max_vocab_length, # set the input shape; size of our vocabulary
                             output_dim=128, # set the size of the embedding vector
                             embeddings_initializer="uniform", # default, initialize embedding vectors randomly
                             input_length=max_length # how long is each input
                             )

embedding

In [32]:
# Get a random sentence from training set
random_sentence = random.choice(train_sentences)
print(f"Original text:\n{random_sentence}\
        \n\nEmbedded version:")

# Embed the random sentence (turn it into numerical representation, aka tokenization first)
sample_embed = embedding(text_vectorizer([random_sentence]))
sample_embed

Original text:
Why is #GOP blocking chance for #DisabledVeterans w/ groin injuries to have children? #ThePartyofMeanness http://t.co/gzTolLl5WoÛ_        

Embedded version:


<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[ 0.03144196,  0.03312844, -0.02113782, ...,  0.00702443,
          0.03811603,  0.01089103],
        [-0.03956036, -0.0224651 , -0.02514894, ..., -0.01409173,
          0.04211445, -0.02801688],
        [-0.00199474,  0.04805091,  0.00267748, ...,  0.04901147,
          0.01839032, -0.04321122],
        ...,
        [ 0.01902029, -0.02009181, -0.02804621, ...,  0.04852505,
         -0.01323269,  0.03549496],
        [-0.02477257, -0.035427  , -0.04802548, ..., -0.00602832,
         -0.0377651 , -0.04274236],
        [-0.0185002 , -0.03050816,  0.00567765, ...,  0.03854169,
          0.01649726, -0.03422958]]], dtype=float32)>

In [33]:
sample_embed[0][0]

<tf.Tensor: shape=(128,), dtype=float32, numpy=
array([ 0.03144196,  0.03312844, -0.02113782,  0.01136748,  0.01253316,
        0.04926391,  0.02810415, -0.00039428,  0.02452599,  0.02481313,
       -0.02367053,  0.04286555,  0.04818505,  0.00070538, -0.04105709,
       -0.01710206,  0.02532817, -0.0036333 ,  0.00358865,  0.03723247,
        0.01895669, -0.00982065, -0.03073506, -0.00034289,  0.02147461,
        0.02345331, -0.03710134, -0.03618769,  0.0353362 , -0.01560335,
        0.04923088, -0.04996717, -0.03853853, -0.01593715, -0.02909789,
        0.04849838, -0.00871726, -0.00860332,  0.01074401,  0.00342969,
        0.00174662, -0.04353111,  0.01875089,  0.04456829, -0.00146132,
        0.03694386,  0.03778498,  0.02120108, -0.03711094,  0.03181418,
       -0.03778942, -0.03746603, -0.00917643, -0.0039205 , -0.03985531,
       -0.04099118, -0.02074867,  0.04539324,  0.04029978, -0.03306577,
       -0.01585995, -0.00289381, -0.03511319,  0.00367439, -0.016     ,
       -0.024119

## Modelling a text dataset (setting up our modelling experiments)

Now we've got our data in numerical format, let's start building and comparing different models.

* Model 0: Naive Bayes (baseline) - got this from here: https://scikit-learn.org/stable/tutorial/machine_learning_map/index.html
* Model 1: Feed-forward neural network (dense model)
* Model 2: LSTM model
* Model 3: GRU model
* Model 4: Bidirectional LSTM
* Model 5: 1D Convolutional Neural Network
* Model 6: TensorFlow Hub Pretrained Word Embedding (feature extractor)
* Model 7: Same as model 6 but using 10% of data

For each of these models, we're going to be following the TensorFlow steps in modelling:
* Construct the model
* Train the model
* Make predictions with the model
* Track prediction evaluation metrics for later comparison

In [34]:
%%time
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

# Create tokenization and modelling pipeline
model_0 = Pipeline([
                    ("tfidf", TfidfVectorizer()), # convert words to numbers using tfidf - https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html
                    ("clf", MultinomialNB()) # model the text converted to numbers
])

# Fit the pipeline to the training data
model_0.fit(train_sentences, train_labels)

CPU times: user 157 ms, sys: 2.68 ms, total: 160 ms
Wall time: 194 ms


Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', MultinomialNB())])

In [35]:
# Let's evalaute our baseline model
baseline_score = model_0.score(val_sentences, val_labels)
print(f"Our baseline model achieves an accuracy of: {baseline_score*100:.2f}%")

Our baseline model achieves an accuracy of: 79.27%


In [36]:
# Make predictions
baseline_preds = model_0.predict(val_sentences)
baseline_preds[:20]

array([1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1])

### Creating an evaluation function for our model experiments

Let's make a function to evaluate our modelling experiment predictions using: 
* Accuracy
* Precision
* Recall
* F1-score

In [37]:
# Function to evalaute: accuracy, precision, recall, F1-score
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def calculate_results(y_true, y_pred):
  """
  Calculates model accuracy, precision, recall and f1 score of a binary classification model.

  Args:
  ----
  y_true = true labels in the form of a 1D array
  y_pred = predicted label in the form of a 1D array

  Returns a dictionary of accuracy, precision, recall and f1-score between y_true and y_pred.
  """
  # Calculate model accuracy
  model_accuracy = accuracy_score(y_true, y_pred) * 100 # get accuracy score in percentage value
  # Calculate model precision, recall and f1 score using "weighted" avergage
  model_precision, model_recall, model_f1, _ = precision_recall_fscore_support(y_true, y_pred, average="weighted")
  # Create a dictionary of model results
  model_results = {"accuracy": model_accuracy,
                   "precision": model_precision,
                   "recall": model_recall,
                   "f1": model_f1}
  return model_results

In [38]:
# Get baseline results
baseline_results = calculate_results(y_true=val_labels,
                                     y_pred=baseline_preds)
baseline_results

{'accuracy': 79.26509186351706,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1': 0.7862189758049549}

### Model 1: A simple dense model (feed-forward neural network)